In [1]:
!pip install face_recognition

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.1/100.1 MB 8.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566170 sha256=f84efd31c9d174a01f549cecc4fb870690f04c78dddfc6c4836f7295383c3927
  Stored in directory: /root/.cache/pip/wheels/7a/eb/cf/e9eced74122b679557f597bb7c8e4c739cfcac526db1fd523d
Successfully built face-recognition-models


In [1]:
#from google.colab import drive
import os
import cv2
#from google.colab.patches import cv2_imshow
import datetime
import csv
import pandas as pd
import face_recognition
from pymongo import MongoClient

In [3]:
# drive.mount('/content/drive', force_remount=True)
# input_folder = '/content/drive/MyDrive/segmentation_to_process'
# output_folder = '/content/drive/MyDrive/segmentation_output'

# now = datetime.datetime.now()
# now.isoformat()
# directory = str(now)

# dest_path = os.path.join(output_folder, directory)
# archive_path = os.path.join(dest_path, "source_files")

# os.mkdir(dest_path)
# os.mkdir(archive_path)
# metadata_path = os.path.join(dest_path, 'metadata.csv')
# header = ['title', 'year', 'issue', 'page', 'filename']

# with open(metadata_path, 'w', encoding='UTF8', newline='') as f:
#     writer = csv.writer(f)

#     # write the header
#     writer.writerow(header)
now = datetime.datetime.now()
now.isoformat()
directory = str(now)
input_folder = 'yearbook'
output_folder = 'yearbook_output'
dest_path = os.path.join(output_folder, directory)
archive_path = os.path.join(dest_path, "source_files")

In [17]:
# Connecting to MongoDB database
client = MongoClient("localhost", 27017)

#database_name = 'peopledata'
#collection_name = 'metadata'

db = client.peopledata
people = db.metadata

In [13]:
def segment(file):
  filebase = os.path.splitext(file)[0]
  try:
    title, year, issue, page,_ = filebase.split('_')
  except:
    print(filebase)
  imagepath = os.path.join(input_folder, file)
  findFaces = face_recognition.load_image_file(imagepath)
  encodings = face_recognition.face_encodings(findFaces)
  if len(encodings) > 0:
    suffix= ".jpg"
    locations = face_recognition.face_locations(findFaces)
    for (top, right, bottom, left) in locations: #we can turn this into multiple little pictures if thats more convinient later
      cv2.rectangle(findFaces, (left, top), (right, bottom), (0, 255, 0), 2)
    dest_filepath = os.path.join(output_folder, filebase + suffix)
    cv2.imwrite(dest_filepath, findFaces)
    #data=[title, year, issue, page, filebase,encodings]
    #df.loc[len(df)] = data
    encodings = [encoding.tolist() for encoding in encodings]
    data = {
        'title': title,
        'year': year,
        'issue': issue,
        'page': page,
        'filename': filebase,
        'encodings': encodings
    }
    # Add a try-except block to catch connection errors
    try:
        people.insert_one(data)
    except Exception as e:
        print(f"Error inserting data into MongoDB: {e}")

In [15]:
# List all .jpg files in the input folder
file_list = [file for file in os.listdir(input_folder) if file.lower().endswith('.jpg')]
print(len(file_list), 'files found')

# Process each file
for file in file_list:
    segment(file)

print("Metadata saved to MongoDB.")

45 files found
Metadata saved to MongoDB.


In [21]:
# Count the number of documents in the 'people' collection
document_count = people.count_documents({})

print(f"Number of documents in the 'people' collection: {document_count}")

Number of documents in the 'people' collection: 28
